# 🔬 Factor Mining & Validation Framework

## 因子挖掘与检验框架教程

本教程演示完整的因子研究流程：

1. **阶段一**: 因子挖掘与初筛 (IC分析、分组回测)
2. **阶段二**: Beta建模与风险因子构建
3. **阶段三**: Alpha因子构建与风险中性化
4. **阶段四**: Fama-MacBeth假设检验


In [ ]:
# 导入必要的库
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# 导入框架模块
from src.factors import NLPFactorBuilder, MicrostructureFactorBuilder, FundamentalFactorBuilder
from src.beta_model import RiskFactorModel
from src.alpha_model import AlphaFactorNeutralizer
from src.backtest import ICAnalyzer, PortfolioBacktest
from src.validation import FamaMacBethRegression
from src.utils import create_sample_data, load_config

print("✓ 模块加载成功!")


## 1. 数据准备

首先生成示例数据用于演示。在实际应用中，您需要替换为真实的市场数据。


In [ ]:
# 生成示例数据
data = create_sample_data(
    n_stocks=100,
    n_days=500,
    start_date="2021-01-01",
    seed=42
)

# 查看数据结构
print("生成的数据集:")
for key, df in data.items():
    print(f"  {key}: {df.shape}")

# 准备数据
returns = data['returns']
nlp_factor = data['nlp_sentiment']

# 转换为长格式
nlp_factor_long = nlp_factor.stack()
nlp_factor_long.name = 'nlp_sentiment'
nlp_factor_long.index.names = ['date', 'stock']
nlp_factor_long = nlp_factor_long.to_frame()

returns_long = returns.stack()
returns_long.name = 'return'
returns_long.index.names = ['date', 'stock']
returns_long = returns_long.to_frame()

print(f"\n因子数据形状: {nlp_factor_long.shape}")
print(f"收益率数据形状: {returns_long.shape}")


---

## 阶段一: 因子挖掘与初筛

### 1.1 IC分析


In [ ]:
# 创建IC分析器
ic_analyzer = ICAnalyzer(
    method='spearman',  # 使用Spearman秩相关
    forward_periods=1,  # 1日前瞻收益
    min_observations=30
)

# 计算IC序列
ic_series = ic_analyzer.compute_ic(
    nlp_factor_long,
    returns_long,
    factor_col='nlp_sentiment',
    return_col='return'
)

# 计算IC统计量
ic_stats = ic_analyzer.compute_ic_stats()

# 打印报告
print(ic_analyzer.generate_report("NLP情绪因子"))


### 1.2 分组回测


In [ ]:
# 创建分组回测器
portfolio = PortfolioBacktest(
    n_groups=5,  # 五分位组合
    holding_period=1,
    weight_method='equal',
    long_short=True
)

# 构建分组组合
group_returns = portfolio.construct_portfolios(
    nlp_factor_long,
    returns_long,
    factor_col='nlp_sentiment',
    return_col='return'
)

# 计算绩效指标
metrics = portfolio.compute_performance_metrics()

# 打印报告
print(portfolio.generate_report("NLP情绪因子"))


---

## 阶段二: Beta建模与风险因子构建

### 2.1 构建Fama-French风险因子 (Formula 1)


In [ ]:
# 创建风险因子模型
risk_model = RiskFactorModel(
    factors=['MKT', 'SMB', 'HML', 'MOM'],
    lookback_window=252
)

# 构建风险因子
factor_returns = risk_model.construct_factors(
    returns=returns,
    market_cap=data['market_cap'],
    book_to_market=data['book_to_market'],
    momentum=data['momentum'],
    rebalance_freq='M'
)

# 风险因子统计
print("风险因子统计:")
print(risk_model.get_factor_statistics())

print("\n风险因子相关性:")
print(risk_model.get_factor_correlations().round(3))


In [ ]:
# 估计滚动Beta
stock_betas = risk_model.estimate_betas(
    stock_returns=returns,
    method='rolling',
    window=126  # 6个月
)

print(f"估计了 {stock_betas.index.get_level_values('stock').nunique()} 只股票的Beta")
print("\nBeta统计:")
print(stock_betas.describe())


---

## 阶段三: Alpha因子构建与风险中性化

### 3.1 因子中性化 (Formula 2)

$$\text{Alpha Factor} = \text{Raw Factor} - \sum \gamma_k \cdot \text{Beta Factor}_k$$


In [ ]:
# 创建中性化器
neutralizer = AlphaFactorNeutralizer(
    neutralize_market=True,
    neutralize_size=True,
    neutralize_industry=True,
    neutralize_beta_factors=True,
    winsorize=True
)

# 准备市值和行业数据
market_cap_long = data['market_cap'].stack()
market_cap_long.name = 'market_cap'
market_cap_long.index.names = ['date', 'stock']
market_cap_long = market_cap_long.to_frame()

industry_long = data['industry'].stack()
industry_long.name = 'industry'
industry_long.index.names = ['date', 'stock']
industry_long = industry_long.to_frame()

# 执行中性化
alpha_factor = neutralizer.neutralize(
    raw_factor=nlp_factor_long,
    beta_factors=stock_betas[[col for col in stock_betas.columns if col.startswith('beta_')]],
    market_cap=market_cap_long,
    industry=industry_long,
    factor_col='nlp_sentiment',
    method='regression'
)

# 打印中性化报告
print(neutralizer.generate_neutralization_report(nlp_factor_long, alpha_factor, stock_betas))


---

## 阶段四: Fama-MacBeth假设检验

### 4.1 Fama-MacBeth回归

$$\text{Ret}_{i,t} - R_{f,t} = \hat{\lambda}_{0,t} + \hat{\lambda}_{\text{Alpha}, t} \cdot \text{Alpha Factor}_{i, t-1} + \sum \hat{\lambda}_{k,t} \cdot \hat{\beta}_{i,k} + \eta_{i,t}$$


In [ ]:
# 创建Fama-MacBeth回归器
fm_regression = FamaMacBethRegression(
    newey_west_lags=6,
    shanken_correction=True,
    min_stocks=30,
    significance_level=0.05
)

# 运行Fama-MacBeth回归
fm_results = fm_regression.run_fama_macbeth(
    returns=returns_long,
    alpha_factor=alpha_factor,
    beta_exposures=stock_betas[[col for col in stock_betas.columns if col.startswith('beta_')]],
    include_intercept=True
)

# 打印详细报告
print(fm_regression.generate_report("NLP情绪Alpha因子"))


---

## 总结

通过这个完整的四阶段流程，我们建立了严谨的证据链来验证因子的有效性：

1. **阶段一**: IC分析和分组回测初步筛选因子
2. **阶段二**: 构建Beta风险因子作为控制变量
3. **阶段三**: 对原始因子进行中性化，提取纯Alpha信号
4. **阶段四**: Fama-MacBeth检验最终验证Alpha因子的定价能力

如果λ_alpha显著异于零，则证明Alpha因子具有独立的、风险调整后的定价能力！
